In [ ]:
import os
import pandas as pd
import rasterio
import rioxarray as rxr
import geopandas as gpd
import numpy as np
from rasterstats import zonal_stats

## Define all paths for the tif files

In [ ]:
main_path = "/Users/jack/Library/CloudStorage/GoogleDrive-limjackailjk@gmail.com/My Drive/Solar PV Lab/NIMBY Project/Jenny's Regression/Data Sources/"

GHI= "GHI-09188ce2.tif"
protected_land = "Protected_Land-5745a356.tif"
habitat= "Habitat-32079c87.tif"
slope= "slope_only-2c1658fa.tif"
popl_dens= "Popl_Density-714f0a64.tif"
Substation = "distance_to_substation_only-f02c9129.tif"
land_cover="Land_Cover-8a2691e6.tif"

# Define the TIF file paths as a list
tif_paths = [GHI, protected_land, habitat, slope, popl_dens, Substation, land_cover]
tif_paths_full = [main_path + path for path in tif_paths]
tif_paths_full

## Load the Counties Bounding Boxes and get the state and county names

In [ ]:
# Load the Rectangular Bounding Box to get the county name and state name
county_bounding_boxes = pd.read_csv('https://gist.githubusercontent.com/a8dx/7e550680f7ea6a68f20da00e21d7ce9b/raw/5f88f425bb362f8e200456fb8181ecb0a8fd55d6/US_County_Boundingboxes.csv')

states_to_drop = ['American Samoa', 'Puerto Rico', 'Alaska', 'Hawaii', 'Commonwealth of the Northern Mariana Islands', 'United States Virgin Islands', 'Guam']

county_bounding_boxes = county_bounding_boxes[~county_bounding_boxes['STATE_NAME'].isin(states_to_drop)]

In [ ]:
# load the full county boxes 
county_bounding_boxes_full = gpd.read_file("/Users/jack/Library/CloudStorage/GoogleDrive-limjackailjk@gmail.com/My Drive/Solar PV Lab/NIMBY Project/scripts/county_box/US County Boundary 2018/cb_2018_us_county_500k.shp")
county_bounding_boxes_full['STATEFP'] = county_bounding_boxes_full['STATEFP'].astype(int)
county_bounding_boxes_full.head()

In [ ]:
# Merge the Files
county_bounding_boxes_full = county_bounding_boxes_full.merge(county_bounding_boxes[['STATE_NAME', 'STATEFP']].drop_duplicates(), left_on='STATEFP', right_on='STATEFP')

county_bounding_boxes_full.head()

In [ ]:
col_names = ["GHI", "Protected_Land", "Habitat", "Slope", "Population_Density", "Distance_to_Substation", "Land_Cover"]

def calculate_zonal_stats(tif_path, geodataframe, nodata_value):
    with rasterio.open(tif_path) as src:
        affine = src.transform
        array = src.read(1)  # Read the first band
        array = np.where(np.isnan(array), nodata_value, array)  # Replace NaNs with nodata_value
        # Debugging: Check raster data and affine transformation
        print(f"Raster data shape: {array.shape}")
        print(f"Affine transformation: {affine}")
        # Check the CRS of the raster
        raster_crs = src.crs
        print(f"Raster CRS: {raster_crs}")
        if geodataframe.crs != raster_crs:
            geodataframe = geodataframe.to_crs(raster_crs)
            
        # filter out values that are less than 100
        array = np.where(array > 101, nodata_value, array)

    # Calculate zonal statistics
    stats = zonal_stats(geodataframe, array, affine=affine, stats="mean", nodata=nodata_value, all_touched=True)
    print(stats)
    # Extract mean values and add to GeoDataFrame
    mean_values = [stat['mean'] for stat in stats]
    return mean_values

def process_tif_files(tif_filepaths, bounding_box, nodata_value=-9999):
    x = bounding_box.copy()

    # Initialize results DataFrame
    results = pd.DataFrame(index=x.index, columns=col_names)

    for tif_path, col_name in zip(tif_filepaths, col_names):
        print(f"Processing {tif_path} for {col_name}")

        # Calculate mean values using zonal stats
        mean_values = calculate_zonal_stats(tif_path, x, nodata_value)

        # Update results DataFrame
        results[col_name] = mean_values

    # Add county and state information
    results["county"] = bounding_box["NAME"]
    results["State"] = bounding_box["STATE_NAME"]

    return results

In [ ]:
# project the bounding box to the same crs as the tif files
county_bounding_boxes_full = county_bounding_boxes_full.to_crs("EPSG:4326")
technoecon_suitability_scores = process_tif_files(tif_paths_full, county_bounding_boxes_full[['geometry', 'NAME', 'STATE_NAME']], nodata_value=np.nan)

In [ ]:
technoecon_suitability_scores[technoecon_suitability_scores['county'] == 'Autauga']

In [ ]:
technoecon_suitability_scores.to_csv('technoecon_suitability_scores.csv', index=False)